## Priprava dat
Rozdeleni dat na testovaci a trenovaco sety: 5krat aby byli co nejvic priblizene k sobe scaffoldy a 5krat aby byli co nejvic vzdalene od sebe scaffoldy

### Molpher
Pro Molpher budou vzaty jen uniaktni csk, takze pak kazdemu unaktnimu scaffoldu bude prirazena nejvic aktivnjesi sloucenina

### Other generators
Pro ostatni generatory budou vzaty vsechny slouceniny

# Library import

In [4]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol
from rdkit.Chem.Scaffolds.MurckoScaffold import MakeScaffoldGeneric
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
from rdkit.DataManip.Metric import GetTanimotoDistMat
from rdkit.DataManip.Metric import GetTanimotoSimMat
from rdkit.Chem import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.ML.Cluster.Butina import EuclideanDist
from rdkit.ML.KNN.DistFunctions import TanimotoDist
from rdkit.DataStructs.cDataStructs import BulkTanimotoSimilarity

#descripts
from rdkit.Chem import Lipinski
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors

from rdkit.Chem.Lipinski import NumAromaticHeterocycles
from rdkit.Chem.Lipinski import NumAliphaticRings
from rdkit.Chem.Lipinski import NumAromaticHeterocycles
from rdkit.Chem.Lipinski import NumAromaticRings

from rdkit.Chem.SaltRemover import SaltRemover

from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

from scipy.spatial import distance
from sklearn import manifold
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import matthews_corrcoef

# plots
from matplotlib import pyplot
import seaborn

import itertools
from itertools import combinations

import numpy as np

from sklearn.metrics import jaccard_score

import os

## Loading data

In [5]:
#Funkce pro opravu problemu z SF5- prevede na CF3
#funkce od Wima
rxn = AllChem.ReactionFromSmarts\
('[*:0][S:1]([F:2])([F:3])([F:4])([F:5])[F:6]>>[*:0]-[C](-[F])(-[F])-[F].[*:1]([*:2])([*:3])([*:4])([*:5])[*:6]')
def MakeScaffoldGeneric_fixed(mol):
    for i in range(len(mol.GetSubstructMatches(Chem.MolFromSmiles("S(F)(F)(F)(F)F")))):
        products = rxn.RunReactants((mol,)) # tuple
        if len(products)>0:
            mol = products[0][0]
    return MakeScaffoldGeneric(mol)

In [40]:
def loading_processing_data(name_file,name):

    dff = pd.read_csv(f"chembl_data/sets/{name_file}/{name}",header=None)
    dff.columns =['molregno', 'stand_type', 
                                    'pchembl_value', 'stand_value',
                                    'canonical_smiles', 'stand_inchi',
                                   'chembl_id', 'tid','pref_name']
            

    dff['scaffolds_csk'] = [1 for x in range(len(dff))]
    delete_element = 0
    for x in range(len(dff)):
                #print(x)
        remover = SaltRemover()
        res = remover.StripMol(Chem.MolFromSmiles(dff['canonical_smiles'][x]))
        dff['canonical_smiles'][x] = Chem.MolToSmiles(res)
        if NumAromaticRings(Chem.MolFromSmiles(dff['canonical_smiles'][x])) == 0 and\
            NumAliphaticRings(Chem.MolFromSmiles(dff['canonical_smiles'][x])) == 0:
            print("Not contain ring")
            print(dff['canonical_smiles'][x])
            display(Chem.MolFromSmiles(dff['canonical_smiles'][x]))
            dff = dff.drop([x])
            delete_element = 1
        if delete_element != 1:
            try:
                dff['scaffolds_csk'].loc[x] = MurckoScaffoldSmiles(\
                                         Chem.MolToSmiles(MakeScaffoldGeneric\
                                       (Chem.MolFromSmiles(dff['canonical_smiles'][x]))))
            except:
                print("Faild to create scaffold_csk")
                print("Index",x)
                print(dff['canonical_smiles'][x])
                display(Chem.MolFromSmiles(dff['canonical_smiles'][x]))
                try:
                    mol = MakeScaffoldGeneric_fixed(Chem.MolFromSmiles(dff['canonical_smiles'][x]))
                    display(mol)
                    dff['scaffolds_csk'].loc[x] = MurckoScaffoldSmiles(Chem.MolToSmiles(mol))
                    display(Chem.MolFromSmiles(MurckoScaffoldSmiles(Chem.MolToSmiles(mol))))
                except:
                    dff = dff.drop([x])
        delete_element = 0
        
    #dff['mfp'] = [AllChem.GetMorganFingerprintAsBitVect\
    #          (Chem.MolFromSmiles(x), 3, nBits = 2048) for x in  dff['scaffolds_csk']]
    dff['mfp'] = [(AllChem.GetMorganFingerprintAsBitVect\
                (Chem.MolFromSmiles(i),3, nBits=2048)) for i in dff['scaffolds_csk']]
      
    dff.reset_index(level=0, inplace=True)
    dff = dff.drop(columns = ['index'])
    return dff

In [41]:
nuclear_glucocor = loading_processing_data('nuclear_threshold_100nM_only_IC50','25_pic50.csv')
#nuclear_glucocor.to_csv('data/input_data/glucocor_recep.csv', index_label = False)
nuclear_glucocor.head()

/tmp/ipykernel_90275/1309756061.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['canonical_smiles'][x] = Chem.MolToSmiles(res)
/tmp/ipykernel_90275/1309756061.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['scaffolds_csk'].loc[x] = MurckoScaffoldSmiles(\


,molregno,stand_type,pchembl_value,stand_value,canonical_smiles,stand_inchi,chembl_id,tid,pref_name,scaffolds_csk,mfp
0,557956,IC50,7.85,14.0,C[C@H]1C[C@@H]2[C@H]3CCC4=CC(=O)C=C[C@@]4(C)[C...,PENABDXTRKXMIW-PZVMULSCSA-N,CHEMBL540936,25,Glucocorticoid receptor,C(CCCCCCC1CCC2C1CCC1C3CCCCC3CCC21)CCCCCC1CCC(C...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1607842,IC50,8.70,2.0,CC[C@@H](Cn1ccc2ccccc21)NS(=O)(=O)c1c(N)cc(Cl)...,RXEIQMFFHIVGLD-AWEZNQCLSA-N,CHEMBL3093461,25,Glucocorticoid receptor,C1CCC(CCCCC2CCC3CCCCC23)CC1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1841315,IC50,7.38,22.0,CC[C@@]12C[C@@](C)(O)[C@](O)(c3cccnc3)C[C@H]1C...,KDUBEHWLJHNXHQ-XUFNMVPLSA-N,CHEMBL3421877,25,Glucocorticoid receptor,C1CCC(C2CCC3C(CCC4CCCCC43)C2)CC1,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,2172692,IC50,7.01,98.0,Cc1cc(C)c(S(=O)(=O)N[C@@H](C)COc2ccc3c(cnn3-c3...,LREULHCSPAWXMB-IBGZPJMESA-N,CHEMBL3963321,25,Glucocorticoid receptor,C1CCC(CCCCCC2CCC3C(CCC3C3CCCCC3)C2)CC1,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,325089,IC50,8.29,5.1,C[C@]12Cc3cnn(-c4ccc(F)cc4)c3C=C1CCC[C@@H]2[C@...,GSBXDPQKEHNRQR-NPAAKHOSSA-N,CHEMBL363179,25,Glucocorticoid receptor,C1CCC(C2CCC3CC4C(CC5CCC6CCCCC65)CCCC4CC32)CC1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Split data to clusters and then create test and train sets

In [107]:
class CustomKMeans(KMeans):
    def __init__(self, n_clusters=5, init='k-means++', 
                 n_init=10, max_iter=300, verbose=False,
                 tol=1e-4, random_state=None, copy_x=True,algorithm='lloyd'
                 ):
        super().__init__(n_clusters=n_clusters, init=init,
                         n_init=n_init, max_iter=max_iter, verbose=verbose,
                         tol=tol, random_state=random_state, copy_x=copy_x,algorithm=algorithm)

    def _transform(self, X):
        return pairwise_distances(X, self.cluster_centers_, metric='wim metric')
    

def split_data_for_clusters(dff):
    df = dff.drop_duplicates(subset='scaffolds_csk', keep="first")
    df.reset_index(level=0, inplace=True)
    df = df.drop(columns = ['index'])
    
    fps = [x for x in df['mfp']]

    for x in range(1):
        kmeans = CustomKMeans(n_clusters=5, random_state = 42)
        kmeans.fit(fps)
        print("Metrics: wim metric")
        print(kmeans.labels_)
        name = 'cluster_' + f"{x}"
        df[name] = kmeans.labels_
    return 

In [108]:
split_data_for_clusters(nuclear_glucocor)

Metrics: wim metric
[3 3 2 4 2 3 2 2 3 3 4 1 4 1 3 3 1 2 1 2 2 1 3 4 4 1 4 0 2 3 1 2 3 3 1 1 1
 1 0 3 4 4 2 1 4 1 4 4 3 2 2 4 2 2 1 3 2 3 3 4 4 2 2 4 0 4 4 4 2 0 3 1 2 1
 1 3 1 2 4 2 3 4 4 1 4 2 3 4 2 2 3 3 3 2 3 4 2 2 2 4 2 4 2 1 3 4 2 4 2 3 0
 0 4 1 3 4 4 3 4 3 0 2 0 3 3 1 4 4 1 1 4 3 4 4 4 4 4 4 1 0 3 2 4 3 2 2 4 4
 4 0 4 3 3 4 4 3 1 0 4 0 1 3 3 4 0 2 2 2 3 0 3 3 1 1 3 3 3 3 2 3 2 4 4 3 1
 4 3 4 4 4 3 2 0 0 3 1 2 4 2 4 3 2 1 4 1 2]


In [109]:
def tanimoto_distance_matrix(fp_list):
    """Calculate distance matrix for fingerprint list"""
    dissimilarity_matrix = []
    # Notice how we are deliberately skipping the first and last items in the list
    # because we don't need to compare them against themselves
    for i in range(1, len(fp_list)):
        # Compare the current fingerprint against all the previous ones in the list
        similarities = DataStructs.BulkTanimotoSimilarity(fp_list[i], fp_list[:i])
        # Since we need a distance matrix, calculate 1-x for every element in similarity matrix
        dissimilarity_matrix.extend([1 - x for x in similarities])
    return dissimilarity_matrix

def newTS(p1,p2):
    p1=DataStructs.CreateFromBitString("".join([str(p) for p in p1]))
    p2=DataStructs.CreateFromBitString("".join([str(p) for p in p2]))
    return 1-DataStructs.TanimotoSimilarity(p1,p2)


def split_data_for_clusters_(dff,cutoff):
    df = dff.drop_duplicates(subset='scaffolds_csk', keep="first")
    df.reset_index(level=0, inplace=True)
    df = df.drop(columns = ['index'])    
    fps = [x for x in df['mfp']]
    # Now cluster the data with the implemented Butina algorithm:
    clusters = Butina.ClusterData(fps, len(fps), cutoff,distFunc = newTS)
    print(clusters)
    print(len(clusters))
    return

def split_data_for_clusters_tsmatrix(dff,cutoff):
    df = dff.drop_duplicates(subset='scaffolds_csk', keep="first")
    df.reset_index(level=0, inplace=True)
    df = df.drop(columns = ['index'])    
    fps = [x for x in df['mfp']]

    # Now cluster the data with the implemented Butina algorithm:
    clusters = Butina.ClusterData(GetTanimotoDistMat(fps), len(fps), cutoff,isDistData=True)
    print(clusters)
    print(len(clusters))
    return


split_data_for_clusters_(nuclear_glucocor,cutoff=0.8)
split_data_for_clusters_tsmatrix(nuclear_glucocor,cutoff=0.8)


((134, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 171, 173, 174, 175, 176, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205), (167, 144), (178, 150, 191), (172,), (149, 27, 38))
5
((134, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1

In [261]:

def split_data_for_clusters_KMedoids(dff):
    df = dff.drop_duplicates(subset='scaffolds_csk', keep="first")
    df.reset_index(level=0, inplace=True)
    df = df.drop(columns = ['index'])
    
    fps = [x for x in df['mfp']]

    kmedoids = KMedoids(n_clusters=5,metric="jaccard", random_state = 1, init='k-medoids++').fit(fps)
    labels = kmedoids.labels_
    print(labels)
    print(len(labels[labels==0]))
    print(len(labels[labels==1]))
    print(len(labels[labels==2]))
    print(len(labels[labels==3]))
    print(len(labels[labels==4]))
    return 

split_data_for_clusters_KMedoids(nuclear_glucocor)

[2 2 1 3 1 2 1 1 2 2 0 0 1 0 2 2 0 1 0 1 1 2 3 3 3 0 3 4 1 2 0 2 2 2 0 0 0
 2 4 2 3 3 1 1 3 0 3 3 2 1 0 4 0 1 4 2 0 2 2 3 3 0 1 3 4 3 3 3 1 4 2 0 1 0
 0 2 0 1 3 2 2 3 3 0 1 0 2 0 1 1 2 2 2 1 2 3 1 2 0 3 1 1 1 2 2 1 1 3 1 2 4
 4 4 0 3 3 3 2 4 3 4 0 4 2 2 0 3 3 0 0 3 2 3 4 3 3 3 3 0 3 2 1 3 2 1 2 3 1
 3 4 3 2 2 3 3 2 2 4 4 3 4 2 3 4 4 2 0 1 2 4 2 3 0 0 2 3 2 2 1 2 1 3 3 2 0
 3 3 4 3 3 2 1 3 4 2 0 1 3 1 2 2 2 2 3 0 1]
35
38
57
54
22


/home/filv/anaconda3/envs/rdkit-env/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:1975: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/filv/anaconda3/envs/rdkit-env/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:1975: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [75]:

from pyclustering.cluster.kmeans import kmeans
from pyclustering.utils.metric import type_metric, distance_metric
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import FCPS_SAMPLES
# Load list of points for cluster analysis.

def user_function():
    print("AAAA")
    p1=DataStructs.CreateFromBitString("".join([str(p) for p in p1]))
    p2=DataStructs.CreateFromBitString("".join([str(p) for p in p2]))
    return 1-DataStructs.TanimotoSimilarity(p1,p2)

def split_data_for_clusters_ntlk(dff):


    sample = read_sample(FCPS_SAMPLES.SAMPLE_TWO_DIAMONDS)
    #print(sample)
    data = [[0,1],[0,2],[0,4]]
    
    initial_centers = kmeans_plusplus_initializer(sample, 2).initialize()
    amount_centers = 4
    amount_candidates = 3
    initializer = kmeans_plusplus_initializer(sample, amount_centers, amount_candidates).initialize()
    print(initial_centers)
    start_centers = [[4.7, 5.9], [5.7, 6.5]]
     metric = distance_metric(type_metric.USER_DEFINED, func=user_function)
    kmeans_instance = kmeans(sample, initial_centers)

    # run cluster analysis and obtain results
    kmeans_instance.process()
    clusters = kmeans_instance.get_clusters()
    
split_data_for_clusters_ntlk(nuclear_glucocor)

AttributeError: module 'numpy' has no attribute 'warnings'

In [110]:
from pyclustering.cluster.kmeans import kmeans, kmeans_visualizer
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.samples.definitions import FCPS_SAMPLES
from pyclustering.utils import read_sample
# Load list of points for cluster analysis.
sample = read_sample(FCPS_SAMPLES.SAMPLE_TWO_DIAMONDS)
# Prepare initial centers using K-Means++ method.
initial_centers = kmeans_plusplus_initializer(sample, 2).initialize()
# Create instance of K-Means algorithm with prepared centers.
kmeans_instance = kmeans(sample, initial_centers)
# Run cluster analysis and obtain results.
kmeans_instance.process()
clusters = kmeans_instance.get_clusters()
final_centers = kmeans_instance.get_centers()
# Visualize obtained results
kmeans_visualizer.show_clusters(sample, clusters, final_centers)

AttributeError: module 'numpy' has no attribute 'warnings'

In [93]:
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import SIMPLE_SAMPLES
sample = read_sample(SIMPLE_SAMPLES.SAMPLE_SIMPLE3)    
# Prepare initial centers - amount of initial centers defines amount of clusters from which X-Means will
# start analysis.
amount_initial_centers = 2
initial_centers = kmeans_plusplus_initializer(sample, amount_initial_centers).initialize()
# Create instance of X-Means algorithm. The algorithm will start analysis from 2 clusters, the maximum
# number of clusters that can be allocated is 20.
xmeans_instance = xmeans(sample, initial_centers, 20)
xmeans_instance.process()
# Extract clustering results: clusters and their centers
clusters = xmeans_instance.get_clusters()
centers = xmeans_instance.get_centers()
# Print total sum of metric errors
print("Total WCE:", xmeans_instance.get_total_wce())
# Visualize clustering results
visualizer = cluster_visualizer()
visualizer.append_clusters(clusters, sample)
visualizer.append_cluster(centers, None, marker='*', markersize=10)
visualizer.show()

AttributeError: module 'numpy' has no attribute 'warnings'

In [254]:
df_clusters_gluco = split_data_for_clusters_(nuclear_glucocor)
#df_clusters_gluco.to_csv('data/input_data/glucocor_recp_split_to_clusters.csv',index_label = False)
df_clusters_gluco.head()

TypeError: split_data_for_clusters_() missing 1 required positional argument: 'cutoff'

In [123]:
df = df_clusters_gluco
print(len(df[df['cluster_0']==1]))
print(len(df[df['cluster_1']==1]))
print(len(df[df['cluster_2']==2]))
print(len(df[df['cluster_3']==3]))
print(len(df[df['cluster_4']==4]))

KeyError: 'cluster_0'

#### Data rozdeleni na shluky
Rozdeleli jsme jednou a ulozili a ted natahujeme ty ulozene data

In [68]:
#nuclear_glucocor = pd.read_csv('data/input_data/glucocor_recep.csv')
nuclear_glucocor.head()
data_clusters_glucocor = pd.read_csv("data/input_data/glucocor_recp_split_to_clusters.csv")
data_clusters_glucocor.head()

,molregno,stand_type,pchembl_value,stand_value,canonical_smiles,stand_inchi,chembl_id,tid,pref_name,scaffolds_csk,mfp,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4
0,557956,IC50,7.85,14.0,C[C@H]1C[C@@H]2[C@H]3CCC4=CC(=O)C=C[C@@]4(C)[C...,PENABDXTRKXMIW-PZVMULSCSA-N,CHEMBL540936,25,Glucocorticoid receptor,C(CCCCCCC1CCC2C1CCC1C3CCCCC3CCC21)CCCCCC1CCC(C...,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,1,1,2,3,4
1,1607842,IC50,8.70,2.0,CC[C@@H](Cn1ccc2ccccc21)NS(=O)(=O)c1c(N)cc(Cl)...,RXEIQMFFHIVGLD-AWEZNQCLSA-N,CHEMBL3093461,25,Glucocorticoid receptor,C1CCC(CCCCC2CCC3CCCCC23)CC1,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,1,1,2,3,4
2,1841315,IC50,7.38,22.0,CC[C@@]12C[C@@](C)(O)[C@](O)(c3cccnc3)C[C@H]1C...,KDUBEHWLJHNXHQ-XUFNMVPLSA-N,CHEMBL3421877,25,Glucocorticoid receptor,C1CCC(C2CCC3C(CCC4CCCCC43)C2)CC1,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,2,0,0,1,0
3,2172692,IC50,7.01,98.0,Cc1cc(C)c(S(=O)(=O)N[C@@H](C)COc2ccc3c(cnn3-c3...,LREULHCSPAWXMB-IBGZPJMESA-N,CHEMBL3963321,25,Glucocorticoid receptor,C1CCC(CCCCCC2CCC3C(CCC3C3CCCCC3)C2)CC1,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0,2,1,1,2
4,325089,IC50,8.29,5.1,C[C@]12Cc3cnn(-c4ccc(F)cc4)c3C=C1CCC[C@@H]2[C@...,GSBXDPQKEHNRQR-NPAAKHOSSA-N,CHEMBL363179,25,Glucocorticoid receptor,C1CCC(C2CCC3CC4C(CC5CCC6CCCCC65)CCCC4CC32)CC1,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,2,0,0,1,2


In [69]:
len(data_clusters_glucocor[data_clusters_glucocor['cluster_1']==1])

55

In [70]:
def chemble_id_for_train_test_sets_dis(data_target,data_cluters, tran, test, cluster):

    input_id_Molpher_tran_nepodobne = []
    input_id_tran_nepodobne = []
    input_id_test_nepodobne = []
    
    count = 0
    for r in tran:
        nazev = "cluster_" + f"{cluster}"
        a = data_cluters[data_cluters[nazev]==r]
        for x in a.index:
            smiles = a['scaffolds_csk'].loc[x]
            b = data_target[data_target['scaffolds_csk']==smiles]
            b = b.sort_values(by=['stand_value'])    
            input_id_Molpher_tran_nepodobne.append(b['chembl_id'].iloc[:1].item())
            for y in b.index:
                input_id_tran_nepodobne.append(b['chembl_id'].loc[y])
    #testovaci_data_target
    nazev = "cluster_" + f"{cluster}"
    a = data_cluters[data_cluters[nazev]==r]
    for x in a.index:
        smiles = a['scaffolds_csk'].loc[x]
        b = data_target[data_target['scaffolds_csk']==smiles]
        b = b.sort_values(by=['stand_value'])    
        for y in b.index:
            input_id_test_nepodobne.append(b['chembl_id'].loc[y])
    
    
    print("Set size for train set for Molpher:",len(input_id_Molpher_tran_nepodobne))
    print("Set size fot train set for other generators:",len(input_id_tran_nepodobne))
    print("Set size for test set:",len(input_id_test_nepodobne))
    return input_id_Molpher_tran_nepodobne,input_id_tran_nepodobne, input_id_test_nepodobne

In [71]:
def sets_for_Molpher(df, data):
    dff = pd.DataFrame(columns = ['start_id','stop_id','start_smiles','stop_smiles'])
    a = list(itertools.permutations(df,2))
    for x in a:
        start_id = x[0]
        stop_id = x[1]
        start_smiles = data[data['chembl_id']==start_id]\
                        ['canonical_smiles'].item()
        stop_smiles = data[data['chembl_id']==stop_id]\
                        ['canonical_smiles'].item()
        dff.loc[len(dff)] = [start_id,stop_id,start_smiles,stop_smiles]
    dff['id'] = [x for x in range(len(dff))]
    new_columns = ['id','start_id','stop_id','start_smiles','stop_smiles']
    dff = dff[new_columns]
    
    return dff
        

In [72]:
def sets_for_other_gen_tran_and_test(df, data):
    dff = pd.DataFrame(columns = ['chembl_id','smiles'])
    for x in df:
        chembl_id = x        
        smiles = data[data['chembl_id']==chembl_id]\
                        ['canonical_smiles'].item()
        
        dff.loc[len(dff)] = [chembl_id,smiles]
    dff['id'] = [x for x in range(len(dff))]
    new_columns = ['id','chembl_id','smiles']
    dff = dff[new_columns]
    
    return dff
        

In [73]:
def split_data_to_train_test_dis(data_target,data_clusters):
    
    for x in ([[1,2,3,4],0,0],[[0,2,3,4],1,1],[[0,1,3,4],2,2],[[0,1,2,4],3,3],[[0,1,2,3],4,4]):
        name = []
        input_id_Molpher_train,input_id_gen_train,input_id_test = chemble_id_for_train_test_sets_dis\
                    (data_target,data_clusters,x[0],x[1],x[2])
        input_Molpher_train = sets_for_Molpher(input_id_Molpher_train, data_target)
        input_gener_train = sets_for_other_gen_tran_and_test\
                        (input_id_gen_train,data_target)
        input_test = sets_for_other_gen_tran_and_test\
                        (input_id_test,data_target)
        
        
        input_Molpher_train.to_csv(f"data/input_data/input_Molpher_glucocor_dis_{x[2]}.csv", index=False)
        input_gener_train.to_csv(f"data/input_data/input_gener_glucocor_dis_{x[2]}.csv", index=False)
        input_test.to_csv(f"data/input_data/input_test_glucocor_dis_{x[2]}.csv",  index=False)

### Create sets for exploration control, name of sets have sufix _dis_

In [74]:
split_data_to_train_test_dis(nuclear_glucocor,data_clusters_glucocor)

Set size for train set for Molpher: 169
Set size fot train set for other generators: 986
Set size for test set: 190
Set size for train set for Molpher: 151
Set size fot train set for other generators: 861
Set size for test set: 71
Set size for train set for Molpher: 155
Set size fot train set for other generators: 867
Set size for test set: 291
Set size for train set for Molpher: 153
Set size fot train set for other generators: 858
Set size for test set: 172
Set size for train set for Molpher: 158
Set size fot train set for other generators: 891
Set size for test set: 244


In [75]:
df = pd.read_csv("data/input_data/glucocor_recp_split_to_clusters.csv") 

In [81]:
dff = pd.DataFrame()
for x in range(5):
    name = 'cluster_' + str(x)
    zero = len(df[df[name]==0])
    one = len(df[df[name]==1])
    two = len(df[df[name]==2])
    three = len(df[df[name]==3])
    four = len(df[df[name]==4])
    dff[x] = [zero,one,two,three,four]


In [82]:
dff

,0,1,2,3,4
0,37,71,46,13,52
1,56,55,31,69,13
2,71,27,51,39,50
3,12,38,31,53,43
4,30,15,47,32,48
